In [1]:
import numpy as np,pandas as pd
from datetime import date

In [2]:
callingdata = pd.read_excel("D:\Manipal Pro learn\Term1\EDA\Dataset\Call Centre Analysis- Data set\Calling Data.xlsx")
enrollmentdata = pd.read_excel("D:\Manipal Pro learn\Term1\EDA\Dataset\Call Centre Analysis- Data set\Enrollment Data.xlsx")
leadiddata = pd.read_excel("D:\Manipal Pro learn\Term1\EDA\Dataset\Call Centre Analysis- Data set\ID Table.xlsx")
leads = pd.read_excel("D:\Manipal Pro learn\Term1\EDA\Dataset\Call Centre Analysis- Data set\Leads.xlsx")

In [3]:
courseiddata = pd.read_excel("D:\Manipal Pro learn\Term1\EDA\Dataset\Call Centre Analysis- Data set\ID Table.xlsx",sheet_name="Course ID Table")

In [12]:
#lead source conversion ratio
leadsource = pd.merge(enrollmentdata,leads,how='left',left_on='Lead ID',right_on='Lead ID')
leadsource_id_list = leadsource.groupby('Lead Source ID').size().reset_index()
leadsource_id_sort_list = leadsource_id_list.sort_values(by=0,ascending=False).reset_index()
enrollsource = pd.merge(leadsource_id_sort_list,leadiddata,left_on='Lead Source ID',right_on='Source ID')[['Source Name',0]]
enrollsource

,Source Name,0
0,Source Not Available,179
1,Google,104
2,Freshersworld,73
3,Facebook,40
4,Digiglitz,36
5,IntellectAds,34
6,3point14,33
7,Way2sms,29
8,Sparrow,29
9,Null,25


In [13]:
leadsource = leads.groupby('Lead Source ID').size().reset_index()
leadsource = pd.merge(leadsource,leadiddata,left_on='Lead Source ID',right_on='Source ID')
leadsource = leadsource[['Source Name',0]]
leadsource

,Source Name,0
0,3point14,3372
1,Adcanopus,580
2,Affiliatevia,952
3,ARQ,755
4,Ashmit,1583
5,Connecting dots,333
6,Digiglitz,1262
7,Expo,319
8,Facebook,3883
9,Freshersworld,2814


In [14]:
#Leadsource conversion ratio - it is the ratio of enroll-source and leads-source
enrollsource.insert(2,'leadsource',leadsource[0])
enrollsource

,Source Name,0,leadsource
0,Source Not Available,179,3372
1,Google,104,580
2,Freshersworld,73,952
3,Facebook,40,755
4,Digiglitz,36,1583
5,IntellectAds,34,333
6,3point14,33,1262
7,Way2sms,29,319
8,Sparrow,29,3883
9,Null,25,2814


In [15]:
enrollsource['lead source conversion ratio'] = enrollsource[0]/enrollsource['leadsource']
enrollsource

,Source Name,0,leadsource,lead source conversion ratio
0,Source Not Available,179,3372,0.053084
1,Google,104,580,0.179310
2,Freshersworld,73,952,0.076681
3,Facebook,40,755,0.052980
4,Digiglitz,36,1583,0.022742
5,IntellectAds,34,333,0.102102
6,3point14,33,1262,0.026149
7,Way2sms,29,319,0.090909
8,Sparrow,29,3883,0.007468
9,Null,25,2814,0.008884


In [17]:
#Average call(in hours) gap for first call
leadsource = pd.merge(leads[['Lead ID','Recent Enquired Date']],callingdata,how='inner',left_on='Lead ID',right_on='Lead ID')
leadsource['call_date'] = pd.to_datetime(leadsource['call_date'])
leadsource['Recent Enquired Date'] = pd.to_datetime(leadsource['Recent Enquired Date'])
leadsource['call diff'] = leadsource['call_date'] - leadsource['Recent Enquired Date']
df= pd.DataFrame()
for leadid,group in leadsource[['Lead ID','call diff']].groupby('Lead ID'):
    cur_row = {'leadid' : leadid,
                'call time' : min(group['call diff'])
              }
    df =  df.append(cur_row,ignore_index=True)
(df['call time'].sum() / df['leadid'].count())

numpy.timedelta64(20330028219424,'ns')

In [22]:
#alternate
first_call_date = callingdata.groupby(['Lead ID'])['call_date'].min().reset_index()
first_call_date = pd.merge(leads,first_call_date,left_on='Lead ID',right_on='Lead ID')
first_call_date['call diff'] = (first_call_date['Assigned On'] - first_call_date['Recent Enquired Date']).dt.days
first_call_date['call diff'].mean() * 24

20.299870955175184

In [9]:
#Number of leads which were never called
ls = list(set(leads['Lead ID']).difference(set(callingdata['Lead ID'])))
ls

['L16539',
 'L19804',
 'L19465',
 'L12498',
 'L31134',
 'L10252',
 'L20095',
 'L16809',
 'L26761',
 'L14901',
 'L6481',
 'L16356',
 'L38111',
 'L17899',
 'L6861',
 'L101',
 'L39190',
 'L3026',
 'L14650',
 'L9372',
 'L19726',
 'L4943',
 'L25694',
 'L3197',
 'L12476',
 'L39512',
 'L15174',
 'L31000',
 'L35797',
 'L40419',
 'L30951',
 'L35508',
 'L30447',
 'L25445',
 'L20399',
 'L105',
 'L31851',
 'L28334',
 'L24586',
 'L32808',
 'L36182',
 'L25787',
 'L4636',
 'L39318',
 'L3057',
 'L35451',
 'L11495',
 'L28772',
 'L33980',
 'L18650',
 'L8930',
 'L12851',
 'L35778',
 'L34178',
 'L35106',
 'L15518',
 'L12240',
 'L18525',
 'L7488',
 'L23155',
 'L5146',
 'L615',
 'L35356',
 'L24569',
 'L19185',
 'L24343',
 'L37311',
 'L31873',
 'L17467',
 'L16975',
 'L38633',
 'L23298',
 'L13872',
 'L8970',
 'L17634',
 'L23007',
 'L32759',
 'L37390',
 'L27035',
 'L18557',
 'L16078',
 'L18820',
 'L1771',
 'L663',
 'L5424',
 'L39973',
 'L9337',
 'L38298',
 'L27315',
 'L41307',
 'L32474',
 'L22812',
 'L16849',


In [40]:
#lead paid 1lakh or above and average calls made
lakh_paid = pd.merge(enrollmentdata[enrollmentdata['Total'] >= 100000],callingdata,how='left',left_on='Lead ID',right_on='Lead ID')
np.average(lakh_paid.groupby('Lead ID')['Order ID'].count())

14.441176470588236

In [38]:
#Counsellor who got assigned and have not called to max leads
not_called = pd.merge(pd.DataFrame(ls),leads,how='left',left_on=0,right_on='Lead ID')
not_called[['Lead ID','Assigned To']].groupby('Assigned To').count().sort_values(by='Lead ID',ascending = False).head()

,Lead ID
Assigned To,
Tejas,24
Pushpanjali,15
AUTOIT,11
Shabana Taj,10
Pavana S,7
